In [ ]:
import pandas as pd
import pandas_datareader as pdr
from datetime import datetime
import yfinance as yf

# Definir el rango de fechas para la obtención de datos
start_date = datetime(2010, 1, 1)
end_date = datetime(2020, 12, 31)

# Obtener datos históricos de precios de acciones de una empresa específica (por ejemplo, Apple)
symbol = 'AAPL'
stock_data = pdr.get_data_yahoo(symbol, start=start_date, end=end_date)

# Descargar datos de los factores Fama-French (3 factores: mercado, tamaño, ratio book-to-market)
ff_factors = pdr.get_data_famafrench('F-F_Research_Data_Factors', start=start_date, end=end_date)[0]

# Merge de datos de precios de acciones y factores Fama-French
data = pd.merge(stock_data['Adj Close'], ff_factors[['SMB', 'HML']], left_index=True, right_index=True)

# Calcular los rendimientos diarios y los rendimientos de los factores
data['Stock Returns'] = data['Adj Close'].pct_change()
data['Market Returns'] = ff_factors['Mkt-RF'] / 100  # Ajustar a porcentaje
data['Size Factor'] = ff_factors['SMB'] / 100
data['Value Factor'] = ff_factors['HML'] / 100

# Aplicar el modelo Fama-French
data['Excess Returns'] = data['Stock Returns'] - data['Market Returns']

# Regresión lineal para calcular alfa, beta, size premium y value premium
import statsmodels.api as sm

X = sm.add_constant(data[['Market Returns', 'Size Factor', 'Value Factor']])
y = data['Excess Returns']

model = sm.OLS(y, X).fit()

# Imprimir los resultados
print(model.summary())


TypeError: ignored